# NBA Data Science Pipeline

In [73]:
%run ../Components/00_ProjectImports.ipynb
%run ../Components/01_UserAgentInformation.ipynb
%run ../Components/02_TeamAbbreviations.ipynb
%run ../Components/03_AdvancedBoxScoreScrapper.ipynb
%run ../Components/04_PlayerURLScrapper.ipynb
%run ../Components/05_PlayerAdvancedBoxScoreScrapper.ipynb
%run ../Components/06_PlayerRegularBoxScoreScrapper.ipynb
%run ../Components/07_RegularBoxScoreScrapper.ipynb

### Year & Team Lists

In [76]:
# Building a dataframe to collect 14 Years worth of game data for each team
# Building an incremental load approach to avoid running into network and connection issues
# Creating a list to contain the years we are going to cycle through
yearsList =[
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
                '2021',
                '2022',
                '2023',
                '2024', 
            ]
currentYearList =['2025']
# Can these lines be deleted??????
teamList1 = nbaTeamList[0:4]
teamList2 = nbaTeamList[5:9]
teamList3 = nbaTeamList[10:14]
teamList4 = nbaTeamList[15:19]
teamList5 = nbaTeamList[20:24]
teamList6 = nbaTeamList[25:30]

### Regular Box Score Incremental Load (Team Level)

* The subsequent code blocks load the regular box scores for all 30 of the nba teams
* Data is being pulled for a 10 season time period, starting in 2015 and ending in 2024
* The data is being loaded incrementally so that the nba reference call limit is not exceded. The call limit is 30 webpages per minute



In [80]:
# Creating our empty dataframe to store all of this team data
regularTeamBoxScoreFrame = pd.DataFrame()

In [83]:
#Loading our dataframe for the first team list
for team in tqdm(nbaTeamList):
    for years in yearsList:
        boxscoreDF = regularBoxScoreScraper(team,years)
        regularTeamBoxScoreFrame = pd.concat([regularTeamBoxScoreFrame, boxscoreDF])
        time.sleep(3)

100%|██████████| 30/30 [21:19<00:00, 42.64s/it]


In [85]:
regularTeamBoxScoreFrame.shape

(24258, 22)

In [87]:
# Storing our raw team regular box score game logs to the psuedo database
# Storing into the excel file 
regularTeamBoxScoreFrame.to_excel("../DataScience_Pipeline/RawDataFiles/teamRegularBoxScoreGamelogs.xlsx")

In [89]:
regularTeamBoxScoreFrame.head()

,Date,Location,Team,Opponent,Team_Points,Opponent_Points,Points_Scored,FTA,FT_Perc,FG,...,ThreeP,ThreePA,ThreePA_Perc,Offensive_Rebounds,Total_Rebounds,Assists,Steals,Blocks,TurnOvers,Personal_Fouls
0,2014-10-29,@,MIL,108,106,108,214,24,.875,39,...,7,14,.500,3,35,25,9,10,15,25
1,2014-10-31,VS,MIL,81,93,81,174,21,.571,36,...,9,24,.375,15,55,25,10,11,21,19
3,2014-11-01,@,MIL,108,97,108,205,24,.833,34,...,9,22,.409,16,42,18,11,3,28,28
4,2014-11-04,@,MIL,81,87,81,168,7,.857,37,...,7,21,.333,9,39,20,12,5,13,22
5,2014-11-05,VS,MIL,95,86,95,181,20,.850,33,...,3,15,.200,18,50,19,10,3,15,28


### Combined Regular Box Score (Team Level)

In [ ]:
# Define a reusable function for running SQL queries
run_query = lambda query: sqldf(query, globals())

In [ ]:
query0 ='''
SELECT 
rtgA.Date,
rtgA.Team,
rtgA.Opponent,
rtgA.Points_Scored,
rtgA.FTA as FTA_Team,
rtgA.FT_Perc as FT_Perc_Team,
rtgA.FG as FG_Team,
rtgA.FGA as FGA_Team,
rtgA.FG_Perc as FG_Perc_Team,
rtgA.ThreeP as ThreeP_Team,
rtgA.ThreePA as ThreePA_Team,
rtgA.ThreePA_Perc as ThreePA_Perc_Team,
rtgA.Offensive_Rebounds as Offensive_Rebounds_Team,
rtgA.Total_Rebounds as Total_Rebounds_Team,
rtgA.Assists as Assists_Team,
rtgA.Steals as Steals_Team,
rtgA.Blocks as Blocks_Team,
rtgA.TurnOvers as TurnOvers_Team,
rtgA.Personal_Fouls as Personal_Fouls_Team,


rtgV.FTA as FTA_Opponent,
rtgV.FT_Perc as FT_Perc_Opponent,
rtgV.FG as FG_Opponent,
rtgV.FGA as FGA_Opponent,
rtgV.FG_Perc as FG_Perc_Opponent,
rtgV.ThreeP as ThreeP_Opponent,
rtgV.ThreePA as ThreePA_Opponent,
rtgV.ThreePA_Perc as ThreePA_Perc_Opponent,
rtgV.Offensive_Rebounds as Offensive_Rebounds_Opponent,
rtgV.Total_Rebounds as Total_Rebounds_Opponent,
rtgV.Assists as Assists_Opponent,
rtgV.Steals as Steals_Opponent,
rtgV.Blocks as Blocks_Opponent,
rtgV.TurnOvers as TurnOvers_Opponent,
rtgV.Personal_Fouls as Personal_Fouls_Opponent

FROM regularTeamBoxScoreFrame rtgA
LEFT JOIN(
            SELECT 
            Date,
            Team,
            Opponent,
            Points_Scored,
            FTA,
            FT_Perc,
            FG,
            FGA,
            FG_Perc,
            ThreeP,
            ThreePA,
            ThreePA_Perc,
            Offensive_Rebounds,
            Total_Rebounds,
            Assists,
            Steals,
            Blocks,
            TurnOvers,
            Personal_Fouls
            FROM regularTeamBoxScoreFrame
            WHERE Location ='VS'
        ) rtgV
on rtgA.Date = rtgV.Date
and rtgA.Opponent = rtgV.Team
where rtgA.Location ='@'

'''

In [ ]:
# Assigning our new combined Dataframe 11979
result_0 = run_query(query0)
result_0.shape

In [ ]:
# Storing our raw team advacned box score game logs to the psuedo database
# Storing into the excel file 
result_0.to_excel("../DataScience_Pipeline/RawDataFiles/combinedRegularBoxScoreGamelogs.xlsx")

### Advanced Box Score Incremental Load (Team Level)

* The subsequent code blocks load the advanced box scores for all 30 of the nba teams
* Data is being pulled for a 10 season time period, starting in 2015 and ending in 2024
* The data is being loaded incrementally so that the nba reference call limit is not exceded. The call limit is 30 webpages per minute


In [96]:
# Creating our empty dataframe to store all of this team data
advancedTeamBoxScoreFrame = pd.DataFrame()

In [98]:
#Loading our dataframe for the first team list
for team in tqdm(nbaTeamList):
    for years in yearsList:
        boxscoreDF = advancedBoxScoreScraper(team,years)
        advancedTeamBoxScoreFrame = pd.concat([advancedTeamBoxScoreFrame,boxscoreDF])
        time.sleep(3)

100%|██████████| 30/30 [18:35<00:00, 37.17s/it]


In [100]:
advancedTeamBoxScoreFrame.shape

(23958, 20)

In [102]:
# Storing our raw team advacned box score game logs to the psuedo database
# Storing into the excel file 
advancedTeamBoxScoreFrame.to_excel("../DataScience_Pipeline/RawDataFiles/teamAdvancedBoxScoreGamelogs.xlsx")

### Combined Advanced & Regular Box Score (Team Level)

In [123]:
# Combining the two datasets to get the regular and advanced data on one sheet
# Using a pandasql library to perform the data join
run_query = lambda query: sqldf(query, globals())

query0 = '''
SELECT
rg.Date,
rg.Location,
rg.Team,

rg.Opponent,
rg.Team_Points,
rg.Opponent_Points,
rg.FTA,
rg.FT_Perc,
rg.FG,
rg.ThreeP,
rg.ThreePA,
rg.ThreePA_Perc,
rg.Offensive_Rebounds,
rg.Total_Rebounds,
rg.Assists,
rg.Steals,
rg.Blocks,
rg.TurnOvers,
rg.Personal_Fouls,
ag.Offensive_EFG,
ag.Defensive_EFG,
ag.Offensive_TurnOver_Rate,
ag.Defensive_TurnOver_Rate,
ag.Offensive_Rebounding_Rate,
ag.Defensive_Rebounding_Rate,
ag.OffensiveRating,
ag.DefensiveRating,
ag.Pace,
ag.FreeThrowAttemptRate,
ag.ThreePointAttemptRate,
ag.TrueShootingRate
FROM regularTeamBoxScoreFrame rg
LEFT JOIN advancedTeamBoxScoreFrame ag
on rg.Date = ag.Date
and rg.Team = ag.Team

'''

In [125]:
# Executing the query stated above
result_0 = run_query(query0)
result_0.shape

(24258, 31)

In [128]:
# Storing our raw team advacned box score game logs to the psuedo database
# Storing into the excel file 
result_0.to_excel("../DataScience_Pipeline/RawDataFiles/combinedAdvancedRegularBoxScoreGamelogs.xlsx")

### Combined Advanced Box Score (Team Level)

In [ ]:
# Define a reusable function for running SQL queries
run_query = lambda query: sqldf(query, globals())

In [ ]:
query1 = '''
SELECT
    advA.Date,
    advA.Team as Team1,
    advA.Opponent as Team2,
    advA.Team_Score as Team1_Points,
    advA.Opponent_Score as Team2_Points,
    advA.Team_Score + advA.Opponent_Score as Total_Points,
    advA.Pace as GamePace,
    advA.Offensive_EFG as Team1_Offensive_EFG,
    advA.Offensive_TurnOver_Rate as Team1_Offensive_TurnOver_Rate,
    advA.Offensive_Rebounding_Rate as Team1_Offensive_Rebounding_Rate,
    advA.Defensive_Rebounding_Rate as Team1_Defensive_Rebounding_Rate,
    advA.OffensiveRating as Team1_OffensiveRating,
    advA.FreeThrowAttemptRate as Team1_FreeThrowAttemptRate,
    advA.ThreePointAttemptRate as Team1_3PointAttemptRate,
    advV.Team2_Offensive_EFG,
    advV.Team2_Offensive_TurnOver_Rate,
    advV.Team2_Offensive_Rebounding_Rate,
    advV.Team2_Defensive_Rebounding_Rate,
    advV.Team2_OffensiveRating,
    advV.Team2_FreeThrowAttemptRate,
    advV.Team2_3PointAttemptRate
from advancedTeamBoxScoreFrame advA
LEFT JOIN(
            SELECT 
                Date,
                Team as Team1,
                Opponent as Team2,
                Team_Score as Team1_Points,
                Opponent_Score as Team2_Points,
               -- Team_Score + adva.Opponent_Score as Total_Points,
                Pace as GamePace,
                Offensive_EFG as Team2_Offensive_EFG,
                Offensive_TurnOver_Rate as Team2_Offensive_TurnOver_Rate,
                Offensive_Rebounding_Rate as Team2_Offensive_Rebounding_Rate,
                Defensive_Rebounding_Rate as Team2_Defensive_Rebounding_Rate,
                OffensiveRating as Team2_OffensiveRating,
                DefensiveRating as Team2_DefensiveRating,
                FreeThrowAttemptRate as Team2_FreeThrowAttemptRate,
                ThreePointAttemptRate as Team2_3PointAttemptRate
            from advancedTeamBoxScoreFrame
            where Location = 'VS'
            )advV
on  advA.Date = advV.Date
and
    advA.Opponent = advV.Team1
    
where advA.Location = '@';
'''

In [ ]:
# Assigning our new combined Dataframe 11979
result_1 = run_query(query1)
result_1.shape

In [ ]:
# Storing our raw team advacned box score game logs to the psuedo database
# Storing into the excel file 
result_1.to_excel("../DataScience_Pipeline/RawDataFiles/combinedAdvancedBoxScoreGamelogs.xlsx")

### Active Player List Incremental Load

* The subsequent code blocks loads our active player list for all 30 of the nba teams
* A player is considered active if the are on the roster and the end of the most recent season
* The data is being loaded incrementally so that the nba reference call limit is not exceded. The call limit is 30 webpages per minute


In [ ]:
# Step one is building an active player list 
# To do this, we will use the most recent nba season and scrape the team rosters to build our player list
activePlayerList = []

# For our incremental load we will use the team lists defined above

In [ ]:
#Loading our dataframe for the first team list
for team in tqdm(nbaTeamList):
    playerList = playerWebPageLinkScraper(team,'2024')
    for player in playerList:
        activePlayerList.append(player)
    time.sleep(3)

In [ ]:
# We want to store this list into our psuedo database
# before doing so, well need to convert the list into a dataframe
activePlayerdf = pd.DataFrame(activePlayerList, columns=['Player_Webpage'])
# Storing into the excel file 
activePlayerdf.to_excel("../DataScience_Pipeline/RawDataFiles/activePlayerList.xlsx")

### Advanced Box Score Incremental Load (Player Level)

* The subsequent code blocks load the advanced box scores for all active players
* Data is being pulled for a 10 season time period, starting in 2015 and ending in 2024
* The data is being loaded incrementally so that the nba reference call limit is not exceded. The call limit is 30 webpages per minute


In [ ]:
# for every player in our active player list, we need to grab the last 10 years worth of their data
# with 550 players will need to build a function to execute the load

#player load executer function call
def playerLoadExecuter(playerURL):
    '''
    INPUTS:
    playerURL - This is a list of strings input
              - This is the url list that leads to the players web page on nba reference
    OUTPUT:
    playerGamelogFrame - Dataframe contain players game logs
    '''
    playerGamelogFrame = pd.DataFrame()
    years = ['2023','2024']
    for url in tqdm(playerURL):
        for year in years:
            try:
                loadFrame = playerAdvancedBoxScoreScraper(url,year)
                playerGamelogFrame = playerGamelogFrame.append(loadFrame)
                time.sleep(3)
            except:
                #print('Load Failure occured')
                time.sleep(3)
    return playerGamelogFrame

In [ ]:
Result1.shape

In [ ]:
Result1 = playerLoadExecuter(list(activePlayerdf['Player_Webpage']))

In [ ]:
Result1.to_excel("../DataScience_Pipeline/RawDataFiles/PlayerAdvacnedBoxScoreHistory.xlsx")

### Regular Box Score Incremental Load (Player Level)

* The subsequent code blocks load the regular box scores for all active players
* Data is being pulled for a 10 season time period, starting in 2015 and ending in 2024
* The data is being loaded incrementally so that the nba reference call limit is not exceded. The call limit is 30 webpages per minute


In [ ]:
# for every player in our active player list, we need to grab the last 10 years worth of their data
# with 550 players will need to build a function to execute the load

#player load executer function call
def playerRegularBoxScoreLoadExecuter(playerURL):
    '''
    INPUTS:
    playerURL - This is a list of strings input
              - This is the url list that leads to the players web page on nba reference
    OUTPUT:
    playerGamelogFrame - Dataframe contain players game logs
    '''
    playerGamelogFrame = pd.DataFrame()
    years = ['2023','2024']
    for url in tqdm(playerURL):
        for year in years:
            try:
                loadFrame = playerRegularBoxScoreScraper(url,year)
                playerGamelogFrame = playerGamelogFrame.append(loadFrame)
                time.sleep(3)
            except:
                #print('Load Failure occured')
                time.sleep(3)
    return playerGamelogFrame

In [ ]:
Result1r.shape

In [ ]:
Result1r = playerRegularBoxScoreLoadExecuter(list(activePlayerdf['Player_Webpage']))

In [ ]:
Result1r.to_excel("../DataScience_Pipeline/RawDataFiles/PlayerRegularBoxScoreHistory.xlsx")